In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a data modelling expert capable of creating high quality entity-relationship models from denormalised datasets. "
            "You always follow these modeling principles: "
            "You don't overnormalize the model. "
            "You don't use the same name for realtionships connecting different types of entities. "
            "You make sure that all features in the dataset are included in the model. "
            "You make sure there is a one to one mapping between the attributes in the extracted entities and the features in the dataset provided as input. ",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


In [3]:
llm = ChatOpenAI(model="gpt-4")

model_generate = prompt | llm
     

In [4]:
import requests

def get_metadata_from_kaggle_croassant(usr_dataset):
    url = f"https://www.kaggle.com/datasets/{usr_dataset}/croissant/download"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        # iterate over fields in the first record set
        return data["name"], data["description"], ", ".join(field["source"]["extract"]["column"] for field in data['recordSet'][0]['field'])
    else:
        return "" # or throw error?
     

def generate_prompt_for_kaggle_dataset(usr_dataset):
    task_description = "From the list of features in the dataset below, extract a list of entities and relationships with their attributes and map them to the features \
        in the dataset. The attributes don't need to be named after the features in the dataset, but they should be mapped to the corresponding feature name. \
        Give meaningful names to relationships rather than just a pair of entities. \
        In addition to that, for each entity, attribute and relationship, provide the closest schema.org term identified by its uri. \
        The output format should be as follows:  \
        {'entities': [ { 'name': 'the entity name', schema_org_term: 'uri of the closest schema.org term for the entity', 'attributes' : [ { 'name': 'the att name', schema_org_term: 'uri of the closest schema.org term for the attribute', 'mappedTo' : 'the feature name'}, ...]}, 'relationships': [ { 'name': 'the entity name', schema_org_term: 'uri of the closest schema.org term for the relationship', 'from': 'source entity', 'to': 'target entity'}]}  \
        Absolutely no extra text or comments, only json as output. \n\n"
    name, desc, features = get_metadata_from_kaggle_croassant(usr_dataset)
    metadata = f"DATASET NAME: {name} \n\nDATASET DESCRIPTION: {desc} \n\nDATASET FEATURES: {features} "
    prompt = task_description + metadata
    return prompt
     


In [6]:
model_as_text = ""
request = HumanMessage(
    content= generate_prompt_for_kaggle_dataset("shashwatwork/dataco-smart-supply-chain-for-big-data-analysis")
)
for chunk in model_generate.stream({"messages": [request]}):
    #print(chunk.content, end="")
    model_as_text += chunk.content

     

print(model_as_text)
     

# Utility function to visualise a data model
from graphviz import Digraph

def generate_graph_viz_and_render(model_t, filename):
  model = eval(model_t)
  graph = Digraph(engine='neato',node_attr={'shape': 'Mrecord'}, format="png")

  for c in model['entities']:
      attnames = (x['name'] + "\n(sch:" + x['schema_org_term'][18:] + ") " for x in c['attributes'])
      graph.node(c['name'].replace(" ","_")+"_node",   "{ Entity: " + c['name']
                  + "\n(sch:" + c['schema_org_term'][18:] + ") " + "|" + "|".join(attnames) + "}")    #shape='ellipse')

  for r in model['relationships']:
      graph.edge(r['from'].replace(" ","_") + "_node",r['to'].replace(" ","_") + "_node",
                  label=r['name'] + "\n(sch:" + r['schema_org_term'][18:] + ") " , len='6.00')

  graph.render(filename)
     

# Visualise the model generated in the previous test
from IPython.display import Image
filename = "model"
generate_graph_viz_and_render(model_as_text, filename)
Image(filename + ".png")
     


{'entities': [ 
{ 'name': 'Customer', 'schema_org_term': 'http://schema.org/Person', 'attributes' : [ 
{ 'name': 'First Name', 'schema_org_term': 'http://schema.org/givenName', 'mappedTo' : 'Customer Fname' },
{ 'name': 'Last Name', 'schema_org_term': 'http://schema.org/familyName', 'mappedTo' : 'Customer Lname' },
{ 'name': 'Email', 'schema_org_term': 'http://schema.org/email', 'mappedTo' : 'Customer Email' },
{ 'name': 'Segment', 'schema_org_term': 'http://schema.org/audience', 'mappedTo' : 'Customer Segment' },
{ 'name': 'Country', 'schema_org_term': 'http://schema.org/addressCountry', 'mappedTo' : 'Customer Country' },
{ 'name': 'City', 'schema_org_term': 'http://schema.org/addressLocality', 'mappedTo' : 'Customer City' },
{ 'name': 'State', 'schema_org_term': 'http://schema.org/addressRegion', 'mappedTo' : 'Customer State' },
{ 'name': 'Street', 'schema_org_term': 'http://schema.org/streetAddress', 'mappedTo' : 'Customer Street' },
{ 'name': 'Zipcode', 'schema_org_term': 'http://s

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH